## 1. Environment set-up

In [2]:
import os
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2
import supervision as sv
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator

## 2. Loading the dataset

In [4]:
dataset = os.listdir('../../../dataset/SteelDefect/data')
images = []

for paths in sorted(dataset):

  image_bgr = cv2.imread(f'../../../dataset/SteelDefect/data/{paths}')

  image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

  data = {'name': paths,
        'bgr': image_bgr,
        'rgb': image_rgb}

  images.append(data)

print(f"O dataset contém {len(images)} imagens.")

O dataset contém 21 imagens.


## 3. Set-up SAM model

In [4]:
sam_checkpoint = "../../models/trained/sam_vit_h_4b8939.pth"
model_type = "vit_h"

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

mask_generator = SamAutomaticMaskGenerator(
    model=sam,
    points_per_side=32,
    pred_iou_thresh=0.86,
    stability_score_thresh=0.92,
    crop_n_layers=1,
    crop_n_points_downscale_factor=2,
    min_mask_region_area=100)

print(f"Dispositivo utilizado: {device}")

Dispositivo utilizado: cuda:0


In [ ]:
def mean_iou(mask):
  iou = np.array([
      iou['predicted_iou'] for iou in mask
  ])

  return iou.mean()

## 4. Automatic mask generation

In [ ]:
ann_images = []
sam_results = []
iou = np.array([])

mask_ann = sv.MaskAnnotator(color_lookup = sv.ColorLookup.INDEX)

for image in images:
  sam_result = mask_generator.generate(image['image_rgb'])

  iou = np.append(iou, mean_iou(sam_result))
  sam_results.append(sam_result)

  detections = sv.Detections.from_sam(sam_result=sam_result)
  ann_image = mask_ann.annotate(scene=image['image_bgr'], detections=detections)

  cv2.imwrite(f'../steel_output/{image["path"]}', ann_image)

In [ ]:
print(f'iou mean: {iou.mean()}')